In [1]:
from spectate import Spectator, WatchedType

In [2]:
class ListSpectator(Spectator):
    
    def __init__(self, inst):
        super(ListSpectator, self).__init__(inst)
        self._change = {'old': None, 'new': None}
    
    def instance_will_call(self, name, args, kwargs):
        # we know the name will be __setitem__
        # since that's all we're spectating
        index, new = args
        try:
            old = self.inst[index]
        except:
            pass
        else:
            if old != new:
                self._change['old'] = {index: old}
                self._change['new'] = {index: new}
    
    def instance_post_call(self, name, returned):
        o, n = self._change['old'], self._change['new']
        if o and n:
            print("%s : %r -> %r" % (name, o, n))
            self._change['old'] = None
            self._change['new'] = None

In [3]:
WatchedList = WatchedType('WatchedList', list, ListSpectator, '__setitem__')

In [4]:
wl = WatchedList([1, 2, 3])

In [5]:
# does not notify because
# values are the same
wl[0] = 1

In [6]:
# will notify when the
# values are different
wl[0] = 2

__setitem__ : {0: 1} -> {0: 2}
